In [1]:
import numpy as np
import pandas as pd
from astropy.constants import sigma_sb, L_sun
from math import pi, sqrt

import sephi_SNL

In [2]:
# DExoplanet directory:
exoplanets_dir = "~/OneDrive/data/NASA_EA_2022_02_09.csv"

# The length of the header in the exoplanets csv file:
header_length = 116

In [3]:
# Read data
exoplanets = pd.read_csv(exoplanets_dir, skiprows=header_length, 
                         usecols=["pl_name", "hostname", "gaia_id", "sy_snum", "sy_pnum", "discoverymethod", "pl_orbper", "pl_orbsmax", "pl_rade", "pl_bmasse", "pl_dens", "pl_orbeccen", "pl_eqt", "pl_orbincl", "st_teff", "st_tefferr1", "st_tefferr2", "st_rad", "st_raderr1", "st_raderr2", "st_mass", "st_met", "st_lum", "st_lumerr1", "st_lumerr2", "st_logg", "st_age", "st_ageerr1", "st_ageerr2", "sy_dist", "sy_plx", "sy_gaiamag"])
#exculuded "st_spectype" due to csv formatting

In [32]:
print(exoplanets.head())

      pl_name  hostname                       gaia_id  sy_snum  sy_pnum  \
0    11 Com b    11 Com  Gaia DR2 3946945413106333696        2        1   
1    11 UMi b    11 UMi  Gaia DR2 1696798367260229376        1        1   
2    14 And b    14 And  Gaia DR2 1920113512486282240        1        1   
3    14 Her b    14 Her  Gaia DR2 1385293808145621504        1        2   
4  16 Cyg B b  16 Cyg B  Gaia DR2 2135550755683407232        3        1   

   discoverymethod   pl_orbper  pl_orbsmax  pl_rade  pl_bmasse  ...  st_lum  \
0  Radial Velocity   326.03000        1.29      NaN  6165.6000  ...   2.243   
1  Radial Velocity   516.21997        1.53      NaN  4684.8142  ...     NaN   
2  Radial Velocity   185.84000        0.83      NaN  1525.5000  ...   1.763   
3  Radial Velocity  1773.40002        2.93      NaN  1481.0878  ...     NaN   
4  Radial Velocity   798.50000        1.66      NaN   565.7374  ...     NaN   

   st_lumerr1  st_lumerr2  st_logg  st_age  st_ageerr1  st_ageerr2   sy_di

In [33]:
# Get rid of exoplanets whose star's haven't been observed by Gaia
exoplanets.dropna(subset=["gaia_id"], inplace=True)

In [31]:
# Not many of the exoplanets have the stellar age listed. Working out how many have stellar age:
#count_NaN = exoplanets["st_age"].isna().sum()
#count_value = exoplanets["st_age"].notna().sum()

print("Number of exoplanets with no stellar age listed: ", exoplanets["st_age"].isna().sum())
print("Number of exoplanets with stellar age listed: ", exoplanets["st_age"].notna().sum())

Number of exoplanets with no stellar age listed:  2450
Number of exoplanets with stellar age listed:  2284


In [5]:
# A few don't have stellar effective temperature listed (st_teff):
print("Number of exoplanets with no teff listed: ", exoplanets["st_teff"].isna().sum())
print("Number of exoplanets with teff listed: ", exoplanets["st_teff"].notna().sum())

Number of exoplanets with no teff listed:  227
Number of exoplanets with teff listed:  4507


In [10]:
# Some don't have stellar radius listed
print("Number of exoplanets with no st_rad listed: ", exoplanets["st_rad"].isna().sum())
print("Number of exoplanets with st_rad listed: ", exoplanets["st_rad"].notna().sum())

print("Number of exoplanets with st_raderr1 listed: ", exoplanets["st_raderr1"].notna().sum())
print("Number of exoplanets with st_raderr2 listed: ", exoplanets["st_raderr2"].notna().sum())

Number of exoplanets with no st_rad listed:  456
Number of exoplanets with st_rad listed:  4458
Number of exoplanets with st_raderr1 listed:  4305
Number of exoplanets with st_raderr2 listed:  4280


In [ ]:
# Number of stars with st_rad and st_teff?



In [ ]:
# Not many have stellar luminosity listed (but this can be calculated):
print("Number of exoplanets with no L listed: ", exoplanets["st_lum"].isna().sum())
print("Number of exoplanets with L listed: ", exoplanets["st_lum"].notna().sum())

Number of exoplanets with no L listed:  3825
Number of exoplanets with L listed:  909


In [49]:
# Do more have their luminosities listed in Gaia?


5.6703744191844314e-08


In [7]:
# Calc. L, R, T
def calc_st_lum(T,  dT1, dT2, R, dR1, dR2):
    """
    T = stellar effective temperature [K]
    dT1 = positive error
    dT2 = negative error
    R = stellar radius [solar radius]
    dR1 = positive error
    dR2 = negative error
    
    returns stellar luminosity [W], positive error, negative error
    """
    #TODO: convert R to meters
    
    sigma=sigma_sb.value
    L = 4 * pi * sigma * R**2 * T**4
    #TODO: convert to log(solar)
    dL1 = 8*pi*sigma * sqrt(dR1**2 + 4*R**2*T**6*dT1**2)
    #print(dR2**2 + 4*R**2*T**6*dT2)
    dL2 = 8*pi*sigma * sqrt(dR2**2 + 4*R**2*T**6*dT2**2)
    return L, dL1, dL2
    
    
def calc_st_rad(T, dT1, dT2, L, dL1, dL2):
    """
    T = stellar effective temperature
    dT1 = positive error
    dT2 = negative error
    L = stellar luminosity
    dL1 = positive error
    dL2 = negative error
    
    returns stellar radius, positive error, negative error
    """
    R = 0.5*T**(-2) * sqrt( L / (pi*sigma) )
    dR1 = 0
    dR2 = 0
    return R, dR1, dR2

# I don't need stellar radius for SEPHI calculations
def calc_st_teff(R, dR1, dR2, L, dL1, dL2):
    """
    R = stellar radius
    dR1 = positive error
    dR2 = negative error
    L = stellar luminosity
    dL1 = positive error
    dL2 = negative error
    
    returns stellar effective temperature, positive error, negative error
    """
    T = (2*R)**(-2) * ( L / (pi*sigma) )**(0.25)
    dT1 = 0
    dT2 = 0
    return T, dT1, dT2

In [18]:
print(exoplanets.shape[0])

4734


In [43]:
#print(exoplanets.loc[:,"st_teff"])
print(pd.isna(exoplanets.loc[0,"st_teff"]))

False


In [13]:
# Estimate L for stars with T and R listed
for i in range(exoplanets.shape[0]):
    if pd.notna(exoplanets.loc[i,"st_teff"]) and pd.notna(exoplanets.loc[i,"st_rad"]):
        L, dL1, dL2 = calc_st_lum(exoplanets.loc[i,"st_teff"], exoplanets.loc[i,"st_tefferr1"], exoplanets.loc[i,"st_tefferr2"], exoplanets.loc[i,"st_rad"], exoplanets.loc[i,"st_raderr1"], exoplanets.loc[i,"st_raderr2"])
        print(L, dL1, dL2)
        #TODO: convert L to appropriate units [log(Solar)]
        #TODO: calc percentage error in L
        #TODO: calc percentage error in st_lum
        #TODO: compare the two
        #TODO: why are some of these repeating?
        
        #If st_lum is listed as nan or if it is a real value with a larger error than the new value then overwrite the previous values
        if pd.isna(exoplanets.loc[i,"st_lum"]) or ( dL1 < exoplanets.loc[i,"st_lumerr1"] and pd.notna(exoplanets.loc[i,"st_lum"]) ): 
            #TODO: overwrite the previous values in exoplanets df
            #exoplanets.loc[i,"st_lum"] = L
            #exoplanets.loc[i, "st_lumerr1"] = dL1
            #exoplanets.loc[i, "st_lumerr2"] = dL2               
            print(L, dL1, dL2)
        else:                                                        
            continue                                                        
    else:
        continue
        

130069359344.14908 577457254.7410556 577457254.7410556
199217939360.44016 292068220.26135623 292068220.26135623
199217939360.44016 292068220.26135623 292068220.26135623
46266901709.52111 69912021.16921385 69912021.16921385
500382111.68881285 10079529.43772612 10079529.43772612
500382111.68881285 10079529.43772612 10079529.43772612
994603232.5299102 4898392.218692902 4898392.218692902
994603232.5299102 4898392.218692902 4898392.218692902
142959063027.07693 58378018.23556005 53070925.668690965
31639379464.095116 nan nan
46238711095.48197 53289313.599159576 53289313.599159576
11556147734.384087 81419764.50355077 81419764.50355077
11556147734.384087 81419764.50355077 81419764.50355077
46433021.82370213 7132568.636513385 7132568.636513385
4923712.738455527 1744694.9278376542 1744694.9278376542
287141.1509435556 332917.2764562964 332917.2764562964
2275878536.4260764 62208177.80436048 62208177.80436048
2275878536.4260764 62208177.80436048 62208177.80436048
88793532283.68504 310949745.7582023 

In [ ]:
# Compare the unc in L if you calculate if vs the unc in the listed values. 
# If the unc. in the calculated value is smaller, replace the listed value

In [ ]:
# Calculate other parameters
# Estimate T for stars without it listed

In [ ]:
# I could then merge the exoplanets data with the phase-space densities calculated from edr3.
# I would need to use a 